In [2]:
from functions.text_process import *
list_file = list_file_data("Practice_05_data/XML-Coll-withSem/")

dataset = "Practice_05_data/XML-Coll-withSem/"
list_termes = {}
for file_name in list_file:
    list_termes= splitDocs(dataset+file_name,list_termes)


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '../Practice_05_data/XML-Coll-withSem/'

In [9]:
def xml_text_minings(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    dl = list()
    for doc_id, content in list_terms.items():
        text_clean = clean(content, use_stem, use_stopword)
        new_list_terms[doc_id] = text_clean.split()  # Here we create a dictionary of Here we create a dictionary of
        # each docments with its terms
        lt = text_clean.split()
        current_dico = countWord(lt)
        posting_list = countWordIntoDocs(current_dico, doc_id, posting_list)
    return posting_list, new_list_terms, (time.time() - start)

In [10]:
pl,lt,ti = xml_text_minings(list_termes,False,False)

In [11]:
stat = get_statistics(pl,lt)

In [12]:
from functions.weiting_function import *

In [17]:
ltn = smart_ltn(pl, stat['n_doc'],stat['colec_freq'])

9804
